# Attributes normalized by city population

In [5]:
import pandas as pd

dataset_size = 'full'

all_data_path = './../data/norm/'
data_path = f'./../data/norm/{dataset_size}/'

join_all_data_path = './../data/join/'
join_data_path = f'./../data/join/{dataset_size}/'


In [6]:
# read brazil's population by city

pop_ddf = pd.read_csv(f'{all_data_path}/brazil_population_2019.csv',
    sep=',',
    dtype={'city_code': float, 'population': float}
)

pop_ddf = pop_ddf[['city_code', 'population']]

pop_ddf.head()


,city_code,population
0,500025.0,5343.0
1,500060.0,39396.0
2,500190.0,23024.0
3,500200.0,11329.0
4,500210.0,24629.0


In [7]:
# read parana covid history

pr_ddf = pd.read_csv(f'{join_data_path}/snap_parana.csv',
    sep=',',
    dtype={'code': float, 'cases': float}
)

pr_ddf = pr_ddf.rename(columns={'code': 'city_code'})

pr_ddf.head()

,city_code,state,name,cases,deaths,vaccinated
0,410010.0,PR,Abatiá,691.0,10,14288
1,410020.0,PR,Adrianópolis,1053.0,21,13590
2,410030.0,PR,Agudos do Sul,438.0,14,20492
3,410040.0,PR,Almirante Tamandaré,8366.0,323,165265
4,410045.0,PR,Altamira do Paraná,102.0,3,7772


In [8]:

def get_pop(code):
    a = pop_ddf.loc[pop_ddf['city_code'] == code]['population']
    if a.shape[0] == 0:
        return 0
    # print(a)
    return a.iloc[0]

def normalize_row(row):
    if row['cases']:
        pop = get_pop(row['city_code'])
        pop_f = pop / 100_000
        if pop > 0:
            row['cases'] /= pop_f
            row['deaths'] /= pop_f
            row['vaccinated'] /= pop_f
        else:
            row['cases'] = 0
            row['deaths'] = 0
            row['vaccinated'] = 0
    return row

# ddf = pd.merge(pr_ddf, pop_ddf, how='left', on='city_code')

ddf = pr_ddf.apply(normalize_row, axis=1)

# ddf = ddf.set_index('date')

ddf.head()


,city_code,state,name,cases,deaths,vaccinated
0,410010.0,PR,Abatiá,9266.461043,134.102186,191605.203165
1,410020.0,PR,Adrianópolis,0.000000,0.000000,0.000000
2,410030.0,PR,Agudos do Sul,4673.994238,149.397076,218674.634511
3,410040.0,PR,Almirante Tamandaré,7052.595197,272.291208,139319.524881
4,410045.0,PR,Altamira do Paraná,0.000000,0.000000,0.000000


In [9]:
print('saving normalized dfs')
ddf.to_csv(f'{data_path}/norm_covid_parana.csv', index=False)


saving normalized dfs
